## Look for units information and compare response_removal() options

In [ ]:
import obspy 
from obspy import Stream
from obspy import Trace
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import numpy as np
import matplotlib.pyplot as plt

client = Client('KNMI')
start_t = UTCDateTime(2019, 1, 1, 0, 0, 0, 0)
print(start_t)
cat = client.get_events(minmagnitude=2.0, starttime=start_t)
print(len(cat))
for e in cat:
    print('event:\n',e)
    
print('Main Event:\n',cat[7])
event = cat[7]
print('Event time = ', event.origins[0].time)

station_str = 'G104'
inv = client.get_stations(network="NL", station=station_str, level="response")
print(type(inv))

print(inv[0].code)
stz = Stream()
for network in inv:
    print('network:\n',network)
    for station in network:
        print('station:\n',station)
        t1 = event.origins[0].time
        t2 = t1 + 16.384
        stz += client.get_waveforms(network.code, station.code, "*", "HHZ", t1, t2, attach_response = True)

print("Stream:\n",stz.traces)

tr_w  = stz[0]
print("Trace.stats:\n",tr_w.stats)
print("Trace:\n",tr_w.stats.response)
tr_wo_d = tr_w.copy()
tr_wo_v = tr_w.copy()
tr_wo_a = tr_w.copy()
tr_wo_d.remove_response(output="DISP")
tr_wo_v.remove_response(output="VEL")
tr_wo_a.remove_response(output="ACC")
tr_wo_d.detrend(type='demean')
tr_wo_v.detrend(type='demean')
tr_wo_a.detrend(type='demean')
tr_wo_d.filter('bandpass',freqmin=2.0,freqmax=6.0,corners=4,zerophase=True)
tr_wo_v.filter('bandpass',freqmin=2.0,freqmax=6.0,corners=4,zerophase=True)
tr_wo_a.filter('bandpass',freqmin=2.0,freqmax=6.0,corners=4,zerophase=True)

nplt = 4
fig, ax = plt.subplots(nplt,figsize=(14,nplt*4.8))
fig.tight_layout()
data_w = tr_w.data
data_wo_d = tr_wo_d.data
data_wo_v = tr_wo_v.data
data_wo_a = tr_wo_a.data
dt     = tr_w.stats.delta
print('dt:',dt)
nt     = len(data_w)
print('nt:',nt)
t      = dt*np.arange(nt)
ax[0].plot(t, data_w,c='black')
ax[1].plot(t, data_wo_d,c='blue')
ax[2].plot(t, data_wo_v,c='orange')
ax[3].plot(t, data_wo_a,c='navy')

plt.show()

## look at the response function of several stations

In [ ]:
import obspy 
from obspy import Stream
from obspy import Trace
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import numpy as np
import matplotlib.pyplot as plt

inv_resp = inv[0][0][0].response
print(vars(inv_resp))

tr_resp = tr_w.stats.response

inv_fig = inv_resp.plot(0.001, output="VEL") 
print(type(inv_fig))

tr_fig = tr_resp.plot(0.001, output="VEL") 

inv_ax = inv_fig.get_axes()
print('inv_ax:',inv_ax[0])
inv_lines0 = inv_ax[0].lines
inv_lines1 = inv_ax[1].lines
ix00 = inv_lines0[0].get_xdata()
ix01 = inv_lines0[1].get_xdata()
ix10 = inv_lines1[0].get_xdata()
ix11 = inv_lines1[1].get_xdata()
print('ix00:',ix00)
print('ix01:',ix01)
print('ix10:',ix10)
print('ix11:',ix11)
print('ix00 - ix10:', ix00-ix10)



tr_ax = tr_fig.get_axes()
print('tr_ax:',tr_ax[0])
tr_lines0 = tr_ax[0].lines
tr_lines1 = tr_ax[1].lines
tx00 = tr_lines0[0].get_xdata()
tx01 = tr_lines0[1].get_xdata()
tx10 = tr_lines1[0].get_xdata()
tx11 = tr_lines1[1].get_xdata()
print('tx00:',tx00)
print('tx01:',tx01)
print('tx10:',tx10)
print('tx11:',tx11)
print('tx00 - tx10:', tx00-tx10)

print()
print('ix00 - tx00:', ix00-tx00)

## Get multiple traces and compare

In [ ]:
import obspy 
from obspy import Stream
from obspy import Trace
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import numpy as np
import matplotlib.pyplot as plt

station_str = 'G104,G103,G234,G233'
inv2 = client.get_stations(network="NL", station=station_str, level="response")
print(type(inv2))


print(inv2[0].code)
stz2 = Stream()
for network in inv2:
    print('network:\n',network)
    for station in network:
        print('station:\n',station)
        t1 = event.origins[0].time
        t2 = t1 + 16.384
        stz2 += client.get_waveforms(network.code, station.code, "*", "HHZ", t1, t2, attach_response=False)

print("Stream:\n",stz2)
inv2.plot_response(0.001,output='DISP')

In [ ]:
#resp0 = stz2[0].stats.response
tr0 = stz2[0]
tr0.attach_response(inv2)
resp0 = tr0.stats.response
for t in range(len(stz2)):
    tr = stz2[t]
    tr.attach_response(inv2)
    #respt = stz2[t].stats.response
    respt = tr.stats.response
    print(respt)
    #print('response[0] == response[%d] ?', resp0 == respt )
    #respt.plot(0.001, output="DISP")